<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<h2>Model Discovery & Tuning of the Radius Neighbors Regressor</h2>
<h3>DAT-5303 | Machine Learning</h3>

<br>
Team Flashd (X): Floris van Splunter, Lorena Cruz, C. L. Alexander Wiese, Saniya Paddalwar, Hiroki Sakai, & Davide Romei

<h2>Loading the Dataset</h2>
<br>

In [1]:
# Importing libraries
import pandas as pd                   # Data science essentials

In [2]:
# Setting pandas print options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Specifying file paths
housing = './datasets/housing_feature_rich.xlsx'

# Reading the file into Python
df = pd.read_excel(io        = housing,
                   header    = 0,
                   index_col = "property_id")

df.head(n = 3) # Checking data

,Lot_Area,Mas_Vnr_Area,Total_Bsmt_SF,First_Flr_SF,Second_Flr_SF,Gr_Liv_Area,Garage_Area,Porch_Area,Sale_Price,m_Lot_Area,m_Mas_Vnr_Area,m_Second_Flr_SF,log_Sale_Price,log_Lot_Area,log_Gr_Liv_Area,has_Second_Flr,has_Garage,has_Mas_Vnr,has_Porch,ratio_building_lot,Overall_Qual,Overall_Cond,Full_Bath,Half_Bath,Kitchen_AbvGr,TotRms_AbvGr,Fireplaces,Garage_Cars,Total_Bath,CulDSac,BrkSide,CollgCr,Crawfor,Edwards,Gilbert,Mitchel,NAmes,NWAmes,NridgHt,OldTown,Sawyer,SawyerW,Somerst,Other_NH
property_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,31770,112,1080,1656,0,1656,528,272,215000,0,0,0,12.278398,10.366309,7.412764,0,1,1,1,0.077306,6,5,1,0,1,7,2,2,1.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,11622,0,882,896,0,896,730,260,105000,0,0,0,11.561725,9.360741,6.799056,0,1,0,1,0.162278,5,6,1,0,1,5,0,1,1.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,14267,108,1329,1329,0,1329,312,429,172000,0,0,0,12.055256,9.565775,7.192934,0,1,1,1,0.145090,6,6,1,1,1,6,0,1,1.5,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [3]:
# All x-data
x_all = df[list(df.drop(labels  = ["Sale_Price", "log_Sale_Price"],
                     axis    = 1))]

# y-data
y_original = df.loc[ : , "Sale_Price"]
y_log      = df.loc[ : , "log_Sale_Price"]

<h2>Creating the Model Function</h2>
<br>

In [4]:
# Importing relevant packages
import numpy as np                    # Mathematical essentials
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.metrics import mean_squared_error
import warnings

# Suppress warnings
warnings.filterwarnings(action = "ignore")

# Creating a list for results
rnr_results = []

# Defining RadiusNeighborsRegressor Function
def radius_neighbors_regressor(note,
                               x_data,
                               y_data,
                               
                               # optional arguments
                               seed              = 1836,
                               tuning            = False,
                               radius_default    = 8,
                               weights_default   = "uniform",
                               p_default         = 2):
    """
    Model Type: Radius Neighbors Regressor
    
    Parameters:
    - x_data (array-like): Feature dataset.
    - y_data (array-like): Target variable.
    - seed (int, optional): Seed for random state, default is 1836.
    - tuning (bool, optional): Flag to enable hyperparameter tuning, default
      is False.
    - radius_default (float): default radius, important to ensure valid
      results if no tuning is applied.
    - weights_default (string): default weight (uniform or distance)
    - p_default (float): default power parameter, must be above 1.
    
    Model Details:
    - The Radius Neighbors Regressor is a type of instance-based learning or
      non-generalizing learning. It does not attempt to construct a general
      internal model. Instead, it stores instances of the training data.
    - Radius Nearest Neighbors is apparent in various domains, such as
      astronomy, robotics, and materials science. In astronomy, the GriSPy
      Python package enables fast fixed-radius nearest-neighbor lookup for
      large datasets. In robotics, collision detection, and motion planning
      algorithms can benefit from efficient nearest-neighbor search
      techniques. In materials science, the solid-angle-based nearest-neighbor
      algorithm (SANN) offers a simple and computationally efficient method
      for identifying nearest neighbors in 3D images.
    - It was originally designed with KNN to solve regression problems by
      considering the average or the weighted average of the k-nearest
      neighbors within a given radius. The Radius adjustment considers the
      radius around the nearest values. Accordingly, the radius must be big
      enough so that no NaN values result.
    - The Radius Neighbors Regressor mimics the human heuristic of "learning
      from those nearest to us" in a figurative sense.
    - Advantages include simplicity, effectiveness in high dimensional spaces,
      and the model's adaptability to changes in the data. It is particularly
      significant when the data has a non-linear structure.
    - Disadvantages are its sensitivity to the local structure of the data,
      the high computational cost for large datasets, and the need for feature
      scaling due to its distance-based nature.
    
    Model Calculations:
    - The model calculates the output by averaging the values of the nearest
      neighbors within a given radius. It can minimize the variance of
      predictions by considering only neighbors within the radius of each
      query point.
    - It does not have automated feature selection. Features used in the model
      should be carefully selected and preprocessed by the user.
    - Data should be standardized or normalized before using this model
      because it is distance-based. Distance metrics can be skewed if one
      feature has a broader range of values than others.
    - Important hyperparameters include "radius" (defines the range within
      which to search for neighbor points), "weights" (determines how much
      influence neighbors have on the prediction), and "p" (the
      power parameter that determines the type of distance).
    
    Model Results:
    - The model's results do not include coefficients. Though, they provide
      distances and indices of neighbors for each prediction point. They
      should be interpreted as the aggregation of the target values of the
      neighbors within the radius. This aggregation provides a
      local average as the prediction.
    - R-squared, AIC, and BIC can be used to evaluate the model's
      performance and fit to the data.
      
    Notes:
    - Ensure the radius is large enough so each data point has neighbors and a
      prediction does not lead to NaN when using the function without tuning.
      
    - If tuning is selected, the function will likely result in many warnings
      due to the predicted NaN values when tuning: randomly selected
      parameters are used together, where a given radius may conflict with
      the given power level, affecting the distance calculations in a way
      that no neighbors are found.
    """

    # Splitting the dataset into training and testing sets
    x_train, x_test, y_train, y_test = train_test_split(x_data,
                                                        y_data,
                                                        test_size    = 0.25,
                                                        random_state = seed)

    # Standardizing the data
    scaler = StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    if tuning:
        
        # Hyperparameter tuning setup
        param_grid = {
            "radius": np.arange(6,50,0.1),
            "weights": ["uniform"], # distance results in overfitting
            "p": np.arange(1,1.5,0.001)
        }
        
        # Define scoring metrics
        scoring = {"R2": "r2",
                   "RMSE": "neg_root_mean_squared_error"}
        
        # Model setup with tuning
        model = RandomizedSearchCV(estimator           = RadiusNeighborsRegressor(),
                                   param_distributions = param_grid,
                                   cv                  = 5,
                                   n_iter              = 200,
                                   scoring             = scoring,
                                   refit               = "R2",
                                   random_state        = 1836,
                                   n_jobs              = -1)
        
    else:
        # Model setup without tuning
        model = RadiusNeighborsRegressor(
            radius        = radius_default, # Ensure radius is big enough to avoid NaN values (default = 1.0)
            weights       = weights_default, # Weight used in prediction {"uniform", "distance"}
            algorithm     = "auto", # Algorithm to compute nearest neighbors {"auto", "ball_tree", "kd_tree", "brute"}
            leaf_size     = 30, # Passed to ball_tree or kd_tree, i.e., tree depth (default = 30)
            p             = p_default, # Power parameter
            metric        = "minkowski", # metric to be used
            metric_params = None, # potentially adding extra arguments
            n_jobs        = -1 # utilizes available CPU cores
        )
    
    # Fitting the model
    model.fit(x_train_scaled, y_train)

    # Making predictions
    y_train_pred = model.predict(x_train_scaled)
    y_test_pred = model.predict(x_test_scaled)

    # Scoring R_Square
    r_square_train = model.score(x_train_scaled, y_train).round(4)
    r_square_test = model.score(x_test_scaled, y_test).round(4)
    model_gap = abs(r_square_train - r_square_test).round(4)
    
    # In-Between Calculations
    mse_train = mean_squared_error(y_train, y_train_pred)
    mse_test = mean_squared_error(y_test, y_test_pred)
    n_train, k = x_train_scaled.shape
    n_test = x_test_scaled.shape[0]

    # Scoring AIC/BIC
    aic_train = (n_train * np.log(mse_train) + 2 * k).round(decimals = 1)
    aic_test  = (n_test * np.log(mse_test) + 2 * k).round(decimals = 1)

    bic_train = (n_train * np.log(mse_train) + np.log(n_train) * k).round(decimals = 1)
    bic_test  = (n_test * np.log(mse_test) + np.log(n_test) * k).round(decimals = 1)

    # Dynamically printing results
    print(f"""
    Radius Neighbors Regressor {note}
    R-Square (training set):    {r_square_train}
    R-Square (testing set):     {r_square_test}
    Train-Test Gap:             {model_gap}

    AIC (training set):         {aic_train}
    AIC (testing set):          {aic_test}

    BIC (training set):         {bic_train}
    BIC (testing set):          {bic_test}
    """)
    
    if tuning:
        # Printing parameters
        print("\tBest Parameters found by RandomizedSearchCV:")
        for param, value in model.best_params_.items():
            print(f"\t{param.capitalize()}: {value}")
        
        # Storing results
        rnr_results.append([note, r_square_train, r_square_test, model_gap,
                            aic_train, aic_test, bic_train, bic_test,
                            model.best_params_.get("radius", "NA"),
                            model.best_params_.get("weights", "NA"),
                            model.best_params_.get("p", "NA")])
    else:
        # Printing parameters
        print(f"""\
        Parameter Settings
        Weights:   {model.weights}
        Radius:    {model.radius}
        Power:     {model.p}
        """)
        
        # Storing results
        rnr_results.append([note, r_square_train, r_square_test, model_gap,
                            aic_train, aic_test, bic_train, bic_test,
                            model.radius, model.weights, model.p])

In [5]:
help(radius_neighbors_regressor) # Calling the help function

Help on function radius_neighbors_regressor in module __main__:

radius_neighbors_regressor(note, x_data, y_data, seed=1836, tuning=False, radius_default=8, weights_default='uniform', p_default=2)
    Model Type: Radius Neighbors Regressor
    
    Parameters:
    - x_data (array-like): Feature dataset.
    - y_data (array-like): Target variable.
    - seed (int, optional): Seed for random state, default is 1836.
    - tuning (bool, optional): Flag to enable hyperparameter tuning, default
      is False.
    - radius_default (float): default radius, important to ensure valid
      results if no tuning is applied.
    - weights_default (string): default weight (uniform or distance)
    - p_default (float): default power parameter, must be above 1.
    
    Model Details:
    - The Radius Neighbors Regressor is a type of instance-based learning or
      non-generalizing learning. It does not attempt to construct a general
      internal model. Instead, it stores instances of the training

<h2>Model Testing</h2>
<br>

In [6]:
# Radius Neighbors Regressor with tuning with y_original
radius_neighbors_regressor(note = "with tuning with y_original",
                           x_data                = x_all,
                           y_data                = y_original,
                           seed                  = 1836,
                           tuning                = True)

# Radius Neighbors Regressor without tuning with y_original
radius_neighbors_regressor(note = "without tuning with y_original",
                           x_data                = x_all,
                           y_data                = y_original,
                           seed                  = 1836,
                           tuning                = False)

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

...

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in

<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: in


    Radius Neighbors Regressor with tuning with y_original
    R-Square (training set):    0.7942
    R-Square (testing set):     0.8041
    Train-Test Gap:             0.0099

    AIC (training set):         46368.8
    AIC (testing set):          15250.8

    BIC (training set):         46607.9
    BIC (testing set):          15443.9
    
	Best Parameters found by RandomizedSearchCV:
	Weights: uniform
	Radius: 17.899999999999956
	P: 1.0969999999999893

    Radius Neighbors Regressor without tuning with y_original
    R-Square (training set):    0.6713
    R-Square (testing set):     0.7101
    Train-Test Gap:             0.0388

    AIC (training set):         47397.5
    AIC (testing set):          15538.0

    BIC (training set):         47636.7
    BIC (testing set):          15731.0
    
        Parameter Settings
        Weights:   uniform
        Radius:    8
        Power:     2
        


In [7]:
# Radius Neighbors Regressor with tuning with y_log
radius_neighbors_regressor(note = "with tuning with y_log",
                           x_data                = x_all,
                           y_data                = y_log,
                           seed                  = 1836,
                           tuning                = True)

# Radius Neighbors Regressor without tuning with y_log
radius_neighbors_regressor(note = "without tuning with y_log",
                           x_data                = x_all,
                           y_data                = y_log,
                           seed                  = 1836,
                           tuning                = False)

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

...

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages

/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/neighbors/_regression.py:500: UserWarning: One or more samples have no neighbors within specified radius; predicting NaN.
  warnings.warn(empty_warning_msg)
/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:842: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 136, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 355, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/constantinwiese/anaconda3/lib/python3.11/site-packages


    Radius Neighbors Regressor with tuning with y_log
    R-Square (training set):    0.1726
    R-Square (testing set):     0.1568
    Train-Test Gap:             0.0158

    AIC (training set):         -4165.0
    AIC (testing set):          -1481.4

    BIC (training set):         -3925.9
    BIC (testing set):          -1288.4
    
	Best Parameters found by RandomizedSearchCV:
	Weights: uniform
	Radius: 47.09999999999985
	P: 1.0059999999999993

    Radius Neighbors Regressor without tuning with y_log
    R-Square (training set):    0.6557
    R-Square (testing set):     0.7012
    Train-Test Gap:             0.0455

    AIC (training set):         -6091.7
    AIC (testing set):          -2241.9

    BIC (training set):         -5852.5
    BIC (testing set):          -2048.8
    
        Parameter Settings
        Weights:   uniform
        Radius:    8
        Power:     2
        


In [8]:
# Outputting Radius Neighbors Regressor Results
pd.DataFrame(data = rnr_results,
             columns = ["regression",
                        "r_square_train", "r_square_test",
                        "model_gap",
                        "aic_train", "aic_test",
                        "bic_train", "bic_test",
                        "radius", "weights", "p"])

,regression,r_square_train,r_square_test,model_gap,aic_train,aic_test,bic_train,bic_test,radius,weights,p
0,with tuning with y_original,0.7942,0.8041,0.0099,46368.8,15250.8,46607.9,15443.9,17.9,uniform,1.097
1,without tuning with y_original,0.6713,0.7101,0.0388,47397.5,15538.0,47636.7,15731.0,8.0,uniform,2.000
2,with tuning with y_log,0.1726,0.1568,0.0158,-4165.0,-1481.4,-3925.9,-1288.4,47.1,uniform,1.006
3,without tuning with y_log,0.6557,0.7012,0.0455,-6091.7,-2241.9,-5852.5,-2048.8,8.0,uniform,2.000


Engaging with the hyperparameter tuning process is strongly advised to optimize the model's performance. This approach allows you to delve into your dataset's intricacies and iteratively adjust the parameter grid to minimize the discrepancy between training and testing performance metrics. It's imperative to recognize that the precise adjustment of the power and radius parameters significantly influences the Radius neighbor regressor's efficacy. A mismatch between these parameters can lead to situations where the model fails to identify neighbors, triggering a flood of warnings. Additionally, it's important to note that applying a log transformation to the target variable (y) can adversely affect the model. This transformation alters the distribution and scale of y, potentially skewing the distance calculations and compromising the model's ability to predict new values accurately. Thus, while increasing the radius might seem like a straightforward fix to prevent the absence of neighbors, this action can lead to significantly poorer model performance, as evidenced by deteriorating R-squared values. In conclusion, a nuanced approach that carefully balances the power and radius parameters - while avoiding transformations that distort the target variable's natural scale - is crucial for securing robust and accurate predictions from the Radius Neighbors Regressor. Also, using distance as weight is highly discouraged as this usually results in an overfitted model.